Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [42]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [43]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [44]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [45]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [46]:
#logistic regression with regularization
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # input data, for the traning data, we use placeholder
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables: init wights and bias
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels]))
    bias = tf.Variable(tf.zeros([num_labels]))
    lamda = tf.placeholder(tf.float32)

    # training computation
    logits = tf.matmul(tf_train_dataset, weights) + bias
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
           labels = tf_train_labels, logits = logits)+lamda*tf.nn.l2_loss(weights))
    
    # optimizer
    Optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # predictions for train, valid, test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights)+bias)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights)+bias)

In [47]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # pick offset for random batch
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # generate minibatch
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels, lamda:0.001}
        
        _, l, predict = session.run([Optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if (step%500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predict, batch_labels))
            print("validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.705973
Minibatch accuracy: 9.4%
validation accuracy: 12.3%
Minibatch loss at step 500: 2.914000
Minibatch accuracy: 75.8%
validation accuracy: 75.9%
Minibatch loss at step 1000: 1.842687
Minibatch accuracy: 73.4%
validation accuracy: 78.3%
Minibatch loss at step 1500: 1.325582
Minibatch accuracy: 75.8%
validation accuracy: 79.7%
Minibatch loss at step 2000: 1.132229
Minibatch accuracy: 79.7%
validation accuracy: 80.3%
Minibatch loss at step 2500: 0.801492
Minibatch accuracy: 83.6%
validation accuracy: 81.2%
Minibatch loss at step 3000: 0.946237
Minibatch accuracy: 76.6%
validation accuracy: 81.3%
Test accuracy: 88.2%


In [48]:
#neural network with regularization
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    # input data, for the traning data, we use placeholder
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables: init wights and bias
    # Variables.
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))

    W2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    lamda = tf.placeholder(tf.float32)

    # training computation
    A1 = tf.nn.relu(tf.add(tf.matmul(tf_train_dataset, W1), b1))
    logits = tf.add(tf.matmul(A1, W2), b2)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
           labels = tf_train_labels, logits = logits))+lamda*(tf.nn.l2_loss(W1)+tf.nn.l2_loss(W2))
    
    # optimizer
    Optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # predictions for train, valid, test data
    train_prediction = tf.nn.softmax(logits)
    valid_activ = tf.nn.relu(tf.matmul(tf_valid_dataset, W1)+b1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_activ, W2)+b2)
    test_activ = tf.nn.relu(tf.matmul(tf_test_dataset, W1)+b1)
    test_prediction = tf.nn.softmax(tf.matmul(test_activ, W2)+b2)

In [49]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # pick offset for random batch
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # generate minibatch
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels, lamda:0.001}
        
        _, l, predict = session.run([Optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if (step%500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predict, batch_labels))
            print("validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 660.869263
Minibatch accuracy: 9.4%
validation accuracy: 29.7%
Minibatch loss at step 500: 194.744293
Minibatch accuracy: 81.2%
validation accuracy: 80.7%
Minibatch loss at step 1000: 115.107536
Minibatch accuracy: 78.9%
validation accuracy: 81.8%
Minibatch loss at step 1500: 68.584435
Minibatch accuracy: 82.8%
validation accuracy: 82.3%
Minibatch loss at step 2000: 41.431015
Minibatch accuracy: 85.9%
validation accuracy: 85.0%
Minibatch loss at step 2500: 25.138103
Minibatch accuracy: 89.1%
validation accuracy: 85.8%
Minibatch loss at step 3000: 15.611264
Minibatch accuracy: 84.4%
validation accuracy: 87.1%
Test accuracy: 92.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [50]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # pick offset for random batch
        offset = ((step%10) * batch_size) % (train_labels.shape[0] - batch_size)
        # generate minibatch
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels, lamda:0.001}
        
        _, l, predict = session.run([Optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if (step%500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predict, batch_labels))
            print("validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 679.823608
Minibatch accuracy: 10.2%
validation accuracy: 30.9%
Minibatch loss at step 500: 191.313065
Minibatch accuracy: 99.2%
validation accuracy: 77.0%
Minibatch loss at step 1000: 115.795586
Minibatch accuracy: 99.2%
validation accuracy: 77.0%
Minibatch loss at step 1500: 70.141396
Minibatch accuracy: 99.2%
validation accuracy: 77.0%
Minibatch loss at step 2000: 42.507393
Minibatch accuracy: 99.2%
validation accuracy: 77.2%
Minibatch loss at step 2500: 25.779673
Minibatch accuracy: 100.0%
validation accuracy: 77.8%
Minibatch loss at step 3000: 15.643139
Minibatch accuracy: 100.0%
validation accuracy: 78.3%
Test accuracy: 85.3%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [51]:
#neural network with dropout
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    # input data, for the traning data, we use placeholder
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables: init wights and bias
    # Variables.
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))

    W2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    lamda = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32)

    # training computation
    A1 = tf.nn.relu(tf.add(tf.matmul(tf_train_dataset, W1), b1))
    A1 = tf.nn.dropout(A1, keep_prob)
    logits = tf.add(tf.matmul(A1, W2), b2)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
           labels = tf_train_labels, logits = logits))+lamda*(tf.nn.l2_loss(W1)+tf.nn.l2_loss(W2))
    
    # optimizer
    Optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # predictions for train, valid, test data
    train_prediction = tf.nn.softmax(logits)
    #validation & test should be not dropout
    valid_activ = tf.nn.relu(tf.matmul(tf_valid_dataset, W1)+b1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_activ, W2)+b2)
    test_activ = tf.nn.relu(tf.matmul(tf_test_dataset, W1)+b1)
    test_prediction = tf.nn.softmax(tf.matmul(test_activ, W2)+b2)

In [52]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # pick offset for random batch
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # generate minibatch
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels, lamda:0.001, keep_prob:0.7}
        
        _, l, predict = session.run([Optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if (step%500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predict, batch_labels))
            print("validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 677.348572
Minibatch accuracy: 9.4%
validation accuracy: 33.0%
Minibatch loss at step 500: 198.209396
Minibatch accuracy: 78.1%
validation accuracy: 79.7%
Minibatch loss at step 1000: 116.557693
Minibatch accuracy: 78.1%
validation accuracy: 80.2%
Minibatch loss at step 1500: 69.096596
Minibatch accuracy: 81.2%
validation accuracy: 82.5%
Minibatch loss at step 2000: 41.806065
Minibatch accuracy: 82.0%
validation accuracy: 84.1%
Minibatch loss at step 2500: 25.243937
Minibatch accuracy: 85.2%
validation accuracy: 85.4%
Minibatch loss at step 3000: 15.715542
Minibatch accuracy: 81.2%
validation accuracy: 86.4%
Test accuracy: 92.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [63]:
#neural network with multi-layers & decay learning rate
batch_size = 128
hidden_nodes_1 = 1024
hidden_nodes_2 = 512
hidden_nodes_3 = 128

graph = tf.Graph()
with graph.as_default():
    # input data, for the traning data, we use placeholder
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables: init wights and bias
    # Variables.
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_1], stddev=np.sqrt(2.0 / (image_size * image_size))))
    b1 = tf.Variable(tf.zeros([hidden_nodes_1]))

    W2 = tf.Variable(tf.truncated_normal([hidden_nodes_1, hidden_nodes_2], stddev=np.sqrt(2.0 / hidden_nodes_1)))
    b2 = tf.Variable(tf.zeros([hidden_nodes_2]))
    
    W3 = tf.Variable(tf.truncated_normal([hidden_nodes_2, hidden_nodes_3], stddev=np.sqrt(2.0 / hidden_nodes_2)))
    b3 = tf.Variable(tf.zeros([hidden_nodes_3]))
    
    W4 = tf.Variable(tf.truncated_normal([hidden_nodes_3, num_labels], stddev=np.sqrt(2.0 / hidden_nodes_3)))
    b4 = tf.Variable(tf.zeros([num_labels]))
    
    lamda = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32)
    global_step = tf.Variable(0)  # count the number of steps taken.

    # training computation
    A1 = tf.nn.relu(tf.add(tf.matmul(tf_train_dataset, W1), b1))
    A1 = tf.nn.dropout(A1, keep_prob)
    A2 = tf.nn.relu(tf.add(tf.matmul(A1, W2), b2))
    A2 = tf.nn.dropout(A2, keep_prob)
    A3 = tf.nn.relu(tf.add(tf.matmul(A2, W3), b3))
    A3 = tf.nn.dropout(A3, keep_prob)

    logits = tf.add(tf.matmul(A3, W4), b4)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
           labels = tf_train_labels, logits = logits))+lamda*(tf.nn.l2_loss(W1)+tf.nn.l2_loss(W2)+tf.nn.l2_loss(W3)+tf.nn.l2_loss(W4))
    
    # optimizer
    learning_rate = tf.train.exponential_decay(0.5, global_step, 500, 0.9)
    if(global_step%100 == 0):
        print(global_step)
    Optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    #Optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # predictions for train, valid, test data
    train_prediction = tf.nn.softmax(logits)
    # validation & test should be not dropout
    valid_A1 = tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset, W1), b1))
    valid_A2 = tf.nn.relu(tf.add(tf.matmul(valid_A1, W2), b2))
    valid_activ = tf.nn.relu(tf.add(tf.matmul(valid_A2, W3), b3))
    valid_prediction = tf.nn.softmax(tf.matmul(valid_activ, W4)+b4)
    test_A1 = tf.nn.relu(tf.add(tf.matmul(tf_test_dataset, W1), b1))
    test_A2 = tf.nn.relu(tf.add(tf.matmul(test_A1, W2), b2))
    test_activ = tf.nn.relu(tf.add(tf.matmul(test_A2, W3), b3))
    test_prediction = tf.nn.softmax(tf.matmul(test_activ, W4)+b4)

In [67]:
num_steps = 8001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # pick offset for random batch
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # generate minibatch
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels, lamda:0.001, keep_prob:0.5}
        
        _, l, predict = session.run([Optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if (step%500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predict, batch_labels))
            print("validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.195601
Minibatch accuracy: 7.8%
validation accuracy: 18.4%
Minibatch loss at step 500: 1.532088
Minibatch accuracy: 79.7%
validation accuracy: 84.1%
Minibatch loss at step 1000: 1.276944
Minibatch accuracy: 82.0%
validation accuracy: 84.9%
Minibatch loss at step 1500: 0.954776
Minibatch accuracy: 83.6%
validation accuracy: 86.2%
Minibatch loss at step 2000: 0.917501
Minibatch accuracy: 81.2%
validation accuracy: 86.5%
Minibatch loss at step 2500: 0.801839
Minibatch accuracy: 85.2%
validation accuracy: 87.1%
Minibatch loss at step 3000: 0.865439
Minibatch accuracy: 82.0%
validation accuracy: 87.0%
Minibatch loss at step 3500: 0.621135
Minibatch accuracy: 87.5%
validation accuracy: 87.2%
Minibatch loss at step 4000: 0.663865
Minibatch accuracy: 85.2%
validation accuracy: 87.9%
Minibatch loss at step 4500: 0.488170
Minibatch accuracy: 91.4%
validation accuracy: 88.2%
Minibatch loss at step 5000: 0.593184
Minibatch accuracy: 89.1%
validation accuracy